# Zajem podatkov

Podatke sem zajemal iz spletne strani https://spin3.sos112.si/javno/porocilo/dnevnibilten. Pri tem pa sem naletel na težavo, da spletna stran deluje tako, da preko `form`-a z `JavaScript`-om pošlje prošnjo na strežnik s podatki in nato prikaže pridobljene podatke. Na brskalniku Chrome sem zato uporabil funkcijo _Inspect_, nato pa na zavihku _Network_ spremljal promet, ki ga izvaja stran. Ugotovil sem, da pošilja `POST`-request na naslov https://spin3.sos112.si/javno/PorociloApi/DnevniBilten in razbral kakšne `headers` in `data` uporabi. Nato sem s pomočjo metode `post` iz knjižnice `requests` poslal podobno zahtevo na ta URL, ki pa mi je vrnila neposredno `json` seznam podatkov za vsak dogodek.

In [1]:
# Uvoz uporabljenih knjižnic:
import json
import requests
import time
import csv
import re

In [2]:
# Konstante za POST request:
URL = "https://spin3.sos112.si/javno/PorociloApi/DnevniBilten"
KATEGORIJE_ID = [
    10014,
    10011,
    10012,
    10013,
    10015,
    10016,
    10017,
    10018,
    10019,
    10020,
    10021,
    10022,
    10023,
]
OBCINE_ID = [
    11026516,
    24063526,
    21436437,
    11026524,
    21427624,
    21427632,
    11026532,
    21427659,
    11026559,
    11026567,
    11026575,
    21427667,
    11026583,
    11026591,
    11026605,
    21427675,
    11026621,
    11026630,
    11026648,
    11026656,
    21427683,
    21436445,
    11026664,
    11026672,
    11026699,
    11026702,
    11026729,
    21427691,
    11026737,
    21427705,
    11026745,
    21427713,
    11026753,
    21427721,
    11026761,
    11026770,
    11026788,
    11026796,
    11026800,
    11026818,
    24063461,
    11026826,
    11026834,
    11026842,
    21427730,
    11026869,
    21427748,
    21427756,
    21427764,
    21427772,
    11026885,
    11027709,
    11026893,
    11026907,
    11027717,
    11026915,
    11026923,
    11027725,
    21427799,
    11026931,
    11027733,
    11026940,
    11027741,
    11026958,
    11027750,
    11026966,
    11027768,
    21427802,
    11027776,
    21436453,
    21427829,
    11026974,
    11027784,
    11027792,
    21427837,
    11026982,
    11027008,
    11027806,
    11027814,
    11027016,
    11027822,
    11027024,
    11027849,
    11027032,
    11027857,
    11027059,
    24063470,
    11027865,
    11027067,
    21427845,
    11027075,
    11027873,
    11027083,
    21436461,
    11027881,
    21428019,
    11027890,
    11027091,
    11027903,
    11027105,
    21428027,
    11027911,
    24063518,
    21427853,
    11027113,
    21436470,
    11027121,
    11027130,
    11027920,
    11027148,
    11027938,
    11027946,
    11027954,
    11027156,
    11027962,
    11027164,
    21427861,
    11027172,
    11027989,
    11027199,
    11027997,
    11028004,
    11027202,
    21427870,
    11028012,
    21436488,
    21428035,
    11027229,
    21427888,
    11027237,
    21428043,
    11028039,
    11028047,
    11027245,
    11028055,
    11027253,
    11027261,
    11027270,
    11027288,
    21427896,
    24063488,
    24063453,
    11027296,
    21428051,
    11027326,
    11027318,
    11027300,
    11027334,
    21427900,
    11027342,
    11027369,
    11027377,
    11027385,
    11027393,
    11027407,
    21428060,
    21427918,
    21433632,
    11027415,
    21433659,
    21433667,
    21428078,
    21428086,
    11027423,
    24063496,
    21433675,
    11026877,
    21427926,
    11027431,
    11027440,
    11027458,
    11027466,
    24063500,
    11027474,
    11027482,
    11027504,
    11027512,
    21433683,
    11027539,
    21428264,
    11027547,
    11027555,
    21427934,
    11026613,
    11027563,
    11027571,
    11027580,
    21427942,
    21427969,
    11027598,
    11027601,
    11027610,
    21428094,
    11027628,
    21427977,
    11028063,
    11027636,
    11028071,
    11027644,
    11028080,
    21428108,
    11027652,
    11028098,
    11027679,
    11028101,
    11027687,
    21428124,
    11028128,
    21427985,
    11027695,
    21428116,
    21427993,
]
PODSKUPINE_ID = [
    10100,
    10200,
    10300,
    10400,
    10500,
    10600,
    10800,
    10900,
    11000,
    11100,
    11200,
    11300,
    11400,
    11500,
    11600,
    20100,
    20200,
    20300,
    20400,
    20500,
    20600,
    20700,
    20800,
    20900,
    30100,
    30200,
    30300,
    30400,
    40100,
    40200,
    40300,
    40400,
    40500,
    50100,
    50200,
    50300,
    60100,
    60200,
    60300,
    70100,
    70200,
    70300,
    80100,
    90100,
]
HEADERS = {"content-type": "application/json"}

In [3]:
def pridobi_podatke(
    limit,
    offset,
    datumOd,
    datumDo,
    kategorije=KATEGORIJE_ID,
    obcine=OBCINE_ID,
    dogodki=PODSKUPINE_ID,
):
    """Pošlje `POST Request` na naslov https://spin3.sos112.si/javno/PorociloApi/DnevniBilten
    in vrne seznam slovarjev dogodkov, ki jih strežnik vrne."""
    data = f'{{"limit":{limit},"offset":{offset},"coezrId":{kategorije},"obcinaMID":{obcine},"dogodekPodskupinaId":{dogodki},"datumOd":"{datumOd}","datumDo":"{datumDo}","corsBesedilo":null}}'
    stran = requests.post(
        url=URL,
        data=data,
        headers=HEADERS,
    )
    slovar = stran.json()
    return slovar["value"]["data"]


def pridobi_iz_json(path):
    with open(path, "r", encoding="utf-8") as dat:
        return json.load(dat)


def shrani_v_json(objekt, path):
    """Shrani objekt `objekt` v `JSON` datoteko na naslovu `path`."""
    with open(path, "w", encoding="utf-8") as datoteka:
        json.dump(objekt, datoteka, ensure_ascii=False, indent=4)


**Demonstracija:** V tej celici program poišče zadnje tri dogodke, ki so bili v evidenco vnešeni dne 1. 1. 2022.

In [4]:
print(pridobi_podatke(limit=3, offset=0, datumOd="2022-01-01T00:00:00", datumDo="2022-01-01T23:59:59"))

[{'dogodekId': 40101, 'dogodekNaziv': 'Stanovanjske stavbe', 'dogodekPodskupinaId': 40100, 'dogodekPodskupinaNaziv': 'Požari v objektih', 'dogodekSkupinaId': 40000, 'dogodekSkupinaNaziv': 'POŽARI IN EKSPLOZIJE', 'obcinaMID': 11027733, 'obcinaNaziv': 'KAMNIK', 'nastanekCas': '2022-01-01T22:22:15', 'corsBesedilo': 'Ob 22.24 je na Parmovi ulici v Kamniku gorelo v kuhinji v prvem nadstropju stanovanjskega objekta. Gasilci PGD Kamnik so pogasili goreče elemente v kuhinji, prezračili stanovanje in iz stanovanja iznesli ožgane elemente. Na kraju je bila prisotna policija in o dogodku je bil obveščen tudi dežurni delavec Elektra Ljubljane zaradi poškodovane glavne električne omarice.'}, {'dogodekId': 30101, 'dogodekNaziv': 'prometne nesreče', 'dogodekPodskupinaId': 30100, 'dogodekPodskupinaNaziv': 'Nesreče v cestnem prometu', 'dogodekSkupinaId': 30000, 'dogodekSkupinaNaziv': 'NESREČE V PROMETU', 'obcinaMID': 21427969, 'obcinaNaziv': 'TRZIN', 'nastanekCas': '2022-01-01T22:12:55', 'corsBesedilo'

In [5]:
def zajemi_vse_podatke(
    path,
    datumOd,
    datumDo,
    po_koliko_naenkrat=20000,
    offset=0,
    kategorije=KATEGORIJE_ID,
    obcine=OBCINE_ID,
    dogodki=PODSKUPINE_ID,
):
    """Zajema podatke po kosih dolžine `po_koliko_naenkrat` in jih vse skupaj shrani v `JSON` datoteko na naslovu `path`."""
    seznam = []
    while True:
        time.sleep(10)
        seznamcek = pridobi_podatke(
            limit=po_koliko_naenkrat,
            offset=offset,
            datumOd=datumOd,
            datumDo=datumDo,
            kategorije=kategorije,
            obcine=obcine,
            dogodki=dogodki,
        )
        if not seznamcek:
            break
        seznam.extend(seznamcek)
        offset += po_koliko_naenkrat
    shrani_v_json(seznam, path)

**Demonstracija:** V tej celici program postopoma nabere vseh 36 dogodkov iz dne 1. 1. 2022 v treh korakih po največ 15 in jih shrani v datoteko `demonstracija/demonstracija_zajetih_podatkov.json`. To predvidoma traja slabo minuto, saj so vmes nastavljeni časovni razmaki v izogib pritožbam strežnika.

In [13]:
JSON_PATH_DEMO = "demonstracija/demonstracija_zajetih_podatkov.json"
zajemi_vse_podatke(
    path=JSON_PATH_DEMO,
    datumOd="2022-01-01T00:00:00",
    datumDo="2022-01-01T23:59:59",
    po_koliko_naenkrat=15)

**Komentar:** Dejansko sem pognal naslednjo kodo, da sem zajel vse podatke od leta 2010 do 2022. Zaradi časovne zahtevnosti je zapisana v Markdown formatu, datoteka `zajeti_podatki/json/surovi_podatki.json` pa se na shranjuje na repozitorij, saj po velikosti presega dovoljenih 100MB.

```
zajemi_vse_podatke(
    path="zajeti_podatki/json/surovi_podatki.json",
    datumOd="2010-01-01T00:00:00",
    datumDo="2023-01-01T00:00:00",
    po_koliko_naenkrat=10000
)
```

## Shranjevanje z manjšo porabo prostora
Prva stvar, ki se jo splača narediti, je shraniti imena občin, skupin, podskupin  in dogodkov po ID-jih v ločene datoteke, saj je nesmiselno vedno znova shranjevati ista imena.

In [7]:
SUROVI_PODATKI_JSON_PATH = "zajeti_podatki/json/surovi_podatki.json"

podatki = pridobi_iz_json(SUROVI_PODATKI_JSON_PATH)

def ustvari_pomozni_slovar(
    ime_json_datoteke, ime_id, ime_opis
):
    """Ustvari slovar z indeksi in njihovimi pomeni ter ga shrani v `JSON` datoteko na naslovu `path`"""
    slovar = {}
    for entry in podatki:
        id = str(entry[ime_id])
        if not id in slovar.keys():
            slovar[id] = entry[ime_opis]
    shrani_v_json(
        slovar, ime_json_datoteke
    )

In [8]:
ustvari_pomozni_slovar("zajeti_podatki/json/skupine.json", "dogodekSkupinaId", "dogodekSkupinaNaziv")
ustvari_pomozni_slovar("zajeti_podatki/json/obcine.json", "obcinaMID", "obcinaNaziv")
ustvari_pomozni_slovar("zajeti_podatki/json/podskupine.json", "dogodekPodskupinaId", "dogodekPodskupinaNaziv")
ustvari_pomozni_slovar("zajeti_podatki/json/dogodki.json", "dogodekId", "dogodekNaziv")

In [2]:
def zapisi_csv(slovarji, imena_polj, ime_datoteke):
    '''Iz seznama slovarjev ustvari CSV datoteko z glavo.'''
    with open(ime_datoteke, 'w', newline="", encoding='utf-8') as csv_datoteka:
        writer = csv.DictWriter(csv_datoteka, fieldnames=imena_polj)
        writer.writeheader()
        writer.writerows(slovarji)

In [3]:
def pomozni_csv(json_path, ime_datoteke_csv):
    '''Prebere datoteko na naslovu `json_path` in njeno vsebino pretvori v `csv`,
    ki ga shrani pod imenom `ime_datoteke_csv`'''
    slovar = pridobi_iz_json(json_path)
    zapisi_csv(
        sorted(
            [{"id": id, "ime": ime} for id, ime in slovar.items()],
            key=lambda slovar: slovar["id"] 
        ),
        imena_polj=["id", "ime"],
        ime_datoteke=ime_datoteke_csv
    )

In [11]:
pomozni_csv("zajeti_podatki/json/dogodki.json", "zajeti_podatki/csv/dogodki.csv")
pomozni_csv("zajeti_podatki/json/obcine.json", "zajeti_podatki/csv/obcine.csv")
pomozni_csv("zajeti_podatki/json/podskupine.json", "zajeti_podatki/csv/podskupine.csv")
pomozni_csv("zajeti_podatki/json/skupine.json", "zajeti_podatki/csv/skupine.csv")

Nazadnje pomembne izmed dobljenih podatkov shranimo v eno `csv` datoteko:

In [12]:
slovarji = pridobi_iz_json(SUROVI_PODATKI_JSON_PATH)
stari_kljuci = ["dogodekId", "obcinaMID", "nastanekCas", "corsBesedilo"]
novi_kljuci = ["dogodek", "obcina", "cas", "opis"]
kratki_slovarji = [
    {
        novi_kljuci[index]: str(slovar.get(stari_kljuci[index], "")).replace("\n", " ")
        for index in range(len(stari_kljuci))
    }
    for slovar in slovarji
]
zapisi_csv(
    slovarji=kratki_slovarji,
    imena_polj=novi_kljuci,
    ime_datoteke="zajeti_podatki/csv/podatki.csv"
)

## Pridobitev dodatnih podatkov o občinah

Za svojo analizo bom potreboval tudi podatke o velikostih občin in njihovi pripadnosti pokrajinam oziroma statističnim regijam. V ta namen bom s spletne strani [Wikipedia](https://sl.wikipedia.org/wiki/Seznam_ob%C4%8Din_v_Sloveniji) pobral potrebne podatke.

In [4]:
stran_z_obcinami = "https://sl.wikipedia.org/wiki/Seznam_ob%C4%8Din_v_Sloveniji"
html_strani = requests.get(stran_z_obcinami).text
with open("zajeti_podatki/obcine.html", "w", encoding="utf-8") as dat:
    dat.write(html_strani)

In [44]:
with open("zajeti_podatki/obcine.html", "r", encoding="utf-8") as dat:
    html_obcine = dat.read()

vzorec_bloka = re.compile(
    r'<tr>\s*<td align="left">(.*?)'
    r'</td>\s*</tr>',
    flags=re.DOTALL
)

vzorec_obcine = re.compile(
    r'<a href=.*?title=.*?>(?P<obcina>.+?)</a>.*?'
    r'<td>(?P<povrsina>.+?)</td>.*?'
    r'<td>(?P<prebivalstvo>.+?)</td>.*?'
    r'<td>(.+?)</td>.*?'
    r'<td>(.+?)</td>.*?'
    r'<td>(.+?)</td>.*?'
    r'<td>(?P<pokrajina>.+?)</td>.*?'
    r'<td>(?P<regija>.+?)</td>.*?',
    flags=re.DOTALL
)


def odstrani_znacke(niz):
    while "<" in niz:
        niz = niz[(niz.find(">") + 1) : niz.rfind("<")]
    return niz


def slovar_iz_bloka(blok):
    slovar = vzorec_obcine.search(blok).groupdict()
    for kljuc in slovar:
        slovar[kljuc] = slovar[kljuc].rstrip()
    slovar["prebivalstvo"] = int(slovar["prebivalstvo"].replace(".", ""))
    slovar["povrsina"] = float(odstrani_znacke(slovar["povrsina"]).replace(",", "."))
    return slovar


seznam_slovarjev_obcin = [slovar_iz_bloka(blok.group(0)) for blok in vzorec_bloka.finditer(html_obcine)]
zapisi_csv(
    slovarji=seznam_slovarjev_obcin,
    imena_polj=["obcina", "povrsina", "prebivalstvo", "pokrajina", "regija"],
    ime_datoteke="zajeti_podatki/csv/podatki_o_obcinah.csv"
)